In [19]:
# Import necessary libraries
import pandas as pd



In [20]:
# Load the dataset
df = pd.read_csv('../data/wp_scored_city_articles_by_state.csv')
df



,state,regional_division,article_title,revision_id,article_quality,population
0,alabama,East South Central,"Abbeville, Alabama",1171163550,C,5074296.0
1,alabama,East South Central,"Adamsville, Alabama",1177621427,C,5074296.0
2,alabama,East South Central,"Addison, Alabama",1168359898,C,5074296.0
3,alabama,East South Central,"Akron, Alabama",1165909508,GA,5074296.0
4,alabama,East South Central,"Alabaster, Alabama",1179139816,C,5074296.0
...,...,...,...,...,...,...
21503,wyoming,Mountain,"Wright, Wyoming",1166334449,GA,581381.0
21504,wyoming,Mountain,"Yoder, Wyoming",1171182284,C,581381.0
21505,missouri,West North Central,"Jennings, Missouri",1165751175,C,6177957.0
21506,pennsylvania,Middle Atlantic,"Jefferson Township, Greene County, Pennsylvania",1171582274,C,12972008.0


In [21]:
# Define article quality classes (you can adapt this based on your ORES predictions)
high_quality_classes = ["FA", "GA"]

# Step 4: Calculate total-articles-per-population and high-quality-articles-per-population

# Calculate the number of articles per state
articles_per_state = df.groupby('state')['article_title'].count().reset_index()



In [22]:
# Merge population data with articles data
population_data = df[['state', 'population']].drop_duplicates()
articles_per_state = articles_per_state.merge(population_data, on='state', how='left')



In [29]:
# Calculate articles per capita
articles_per_state['Articles_Per_Capita'] = articles_per_state['article_title'] / articles_per_state['population']

# Calculate high-quality articles per state
high_quality_articles_per_state = df[df['article_quality'].isin(high_quality_classes)].groupby('state')['article_title'].count().reset_index()
articles_per_state


,state,article_title,population,Articles_Per_Capita
0,alabama,461,5074296.0,0.000091
1,alaska,148,733583.0,0.000202
2,arizona,91,7359197.0,0.000012
3,arkansas,500,3045637.0,0.000164
4,california,482,39029342.0,0.000012
5,colorado,288,5839926.0,0.000049
6,delaware,57,1018396.0,0.000056
7,florida,411,22244823.0,0.000018
8,georgia_(u.s._state),538,NaN,NaN
9,hawaii,151,1440196.0,0.000105


In [26]:
# Merge population data with high-quality articles data
high_quality_articles_per_state = high_quality_articles_per_state.merge(population_data, on='state', how='left')

# Calculate high-quality articles per capita
high_quality_articles_per_state['High_Quality_Articles_Per_Capita'] = high_quality_articles_per_state['article_title'] / high_quality_articles_per_state['population']



In [28]:
# Step 5: Generate the required tables

# Top 10 US states by coverage
top_10_states_by_coverage = articles_per_state.sort_values(by='Articles_Per_Capita', ascending=False).head(10)

# Bottom 10 US states by coverage
bottom_10_states_by_coverage = articles_per_state.sort_values(by='Articles_Per_Capita', ascending=True).head(10)

# Top 10 US states by high quality
top_10_states_by_high_quality = high_quality_articles_per_state.sort_values(by='High_Quality_Articles_Per_Capita', ascending=False).head(10)

# Bottom 10 US states by high quality
bottom_10_states_by_high_quality = high_quality_articles_per_state.sort_values(by='High_Quality_Articles_Per_Capita', ascending=True).head(10)

# Census divisions by total coverage
census_divisions_by_total_coverage = articles_per_state.groupby('Regional_Division')['Articles_Per_Capita'].mean().reset_index()
census_divisions_by_total_coverage = census_divisions_by_total_coverage.sort_values(by='Articles_Per_Capita', ascending=False)

# Census divisions by high quality coverage
census_divisions_by_high_quality_coverage = high_quality_articles_per_state.groupby('Regional_Division')['High_Quality_Articles_Per_Capita'].mean().reset_index()
census_divisions_by_high_quality_coverage = census_divisions_by_high_quality_coverage.sort_values(by='High_Quality_Articles_Per_Capita', ascending=False)

# Display the generated tables
print("Top 10 US States by Coverage:")
print(top_10_states_by_coverage)
print("\nBottom 10 US States by Coverage:")
print(bottom_10_states_by_coverage)
print("\nTop 10 US States by High Quality:")
print(top_10_states_by_high_quality)
print("\nBottom 10 US States by High Quality:")
print(bottom_10_states_by_high_quality)
print("\nCensus Divisions by Total Coverage:")
print(census_divisions_by_total_coverage)
print("\nCensus Divisions by High Quality Coverage:")
print(census_divisions_by_high_quality_coverage)

KeyError: 'regional_division'